# Markov Localization

1D landmark based localization 

### Prerequisites

Install jupyter c++ extensions
```
conda install xeus-cling xplot -c QuantStack -c conda-forge
conda install widgetsnbextension -c conda-forge
conda install bqplot -c conda-forge
```

### Problem Definition

Known:
* Observations in local frame (sensor measurements) *$z_{1:t}$*
* Controls of car *$u_{1:t}$*
* Map with landmarks in global coordinate system *m*

Unknown:
* Position of car *$x_t$*

Posterior distribution: $bel(x_t) = p(x_t | z_{1:t}, u_{1:t}, m)$

In [1]:
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm>
#include <numeric>

#include "xwidgets/xbox.hpp"
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"
#include "xplot/xtoolbar.hpp"

using namespace std;

### Helpers

In [2]:
// plot probability
auto plot(vector<float> prob, float max = 1.0) {
    std::vector<double> x(prob.size());
    std::iota(x.begin(), x.end(), 0);

    // memory leak here
    xpl::ordinal_scale* sx = new xpl::ordinal_scale();
    xpl::linear_scale* sy = new xpl::linear_scale();
    sy->min = 0.0;
    sy->max = max;
    
    // make figure
    xpl::figure fig;
    fig.add_mark(
        xpl::bars_generator(*sx, *sy)
        .x(x)
        .y(prob)
        .padding(0.5)
        .colors(std::vector<std::string>({"orange"}))
        .finalize());
    fig.add_axis(xpl::axis_generator(*sx)
        .label("location")
        .finalize());
    fig.add_axis(xpl::axis_generator(*sy)
        .label("probability")
        .orientation("vertical")
        .side("left")
        .finalize());
    return fig;
}

In [3]:
// normalized probability density function
float normpdf(float x, float mu, float std) {
    return (1.f/sqrt(2*M_PI)*std) * exp(-0.5*pow((x-mu)/std, 2));
}

In [4]:
// normalize a vector:
auto normalize_vector(std::vector<float> inputVector) {
    //declare and resize output vector:
    std::vector<float> outputVector(inputVector.size());
    
    //estimate the sum:
    float sum = 0.0f;
    for (unsigned int i = 0; i < inputVector.size(); ++i) {
        sum += inputVector[i];
    }

    // normalize with sum:
    for (unsigned int i = 0; i < inputVector.size(); ++i) {
        outputVector[i] = inputVector[i]/sum;
    }
    return outputVector;
}

### Initial Conditions

In [5]:
// set standard deviation of control
float control_stdev = 1.0f;

// set standard deviation of position
float position_stdev = 1.0f;

// meters vehicle moves per time step
float movement_per_timestep = 1.0f;

// set observation standard deviation
float observation_stdev = 1.0f;

// number of x positions on map
int map_size = 25;

// define landmarks
vector<float> landmark_positions {3, 9, 14, 23};

// define observations vector, each inner vector represents a set 
//   of observations for a time step
vector<vector<float> > sensor_obs {{1,7,12,21}, {0,6,11,20}, {5,10,19},
                                   {4,9,18}, {3,8,17}, {2,7,16}, {1,6,15}, 
                                   {0,5,14}, {4,13}, {3,12}, {2,11}, {1,10},
                                   {0,9}, {8}, {7}, {6}, {5}, {4}, {3}, {2},
                                   {1}, {0}, {}, {}, {}};

### Initialize Priors
Assuming we are parked next to a landmark and our position standard deviationis $+/-x$ (spread of our actual position)

In [6]:
auto initialize_priors(int map_size, vector<float> landmark_positions, 
                       float position_stdev) {
    // set all priors to zero
    vector<float> priors(map_size, 0.0);
    
    // normalizer
    float norm_term = landmark_positions.size() * (position_stdev * 2 + 1);
    
    // set each landmark +/- position_stddev to 1
    for(auto const& landmark: landmark_positions) {
        for(float j=landmark-position_stdev; j<=landmark+position_stdev; j++) {
            priors[int(j) % map_size] += 1. / norm_term;
        }
    }
    
    return priors;
}

In [7]:
auto priors = initialize_priors(map_size, landmark_positions, position_stdev);

xpl::figure fig = plot(priors);
fig

A Jupyter widget

### Define belief as a recursive algorithm

$bel(x_t) = p(x_t | z_{1:t}, u_{1:t}, m)$

$bel(x_t) = p(x_t | z_t, z_{1:t-1}, u_{1:t}, m)$

$bel(x_t) = \frac {p(z_t | x_t, z_{1:t-1}, u_{1:t}, m) \times p(x_t | z_{1:t-1}, u_{1:t}, m)} {p(x_t | z_{1:t-1}, u_{1:t}, m)} = $ Normalizer x Observation Model x Motion Model (Bayes rule)

$\eta = \frac {1}{p(x_t | z_{1:t-1}, u_{1:t}, m)} = \frac{1}{\sum_{i}{p(z_t | x_t^i, z_{1:t-1}, u_{1:t}, m) \times p(x_t^i | z_{1:t-1}, u_{1:t}, m)}}$ (Normalizer)

### Motion Model

$p(x_t | z_{1:t-1}, u_{1:t}, m) = \int {p(x_t | x_{t-1}, z_{1:t-1}, u_{1:t}, m) p(x_{t-1} | z_{1:t-1}, u_{1:t}, m)} dx_{t-1}$ (Law of total probability)

$p(x_t | z_{1:t-1}, u_{1:t}, m) = \int {p(x_t | x_{t-1}, u_t, m) p(x_{t-1} | z_{1:t-1}, u_{1:t-1}, m)} dx_{t-1}$ = Transition Model x Prior (Markov Assumption)

$p(x_t | z_{1:t-1}, u_{1:t}, m) = \int {p(x_t | x_{t-1}, u_t, m) bel(x_{t-1})} dx_{t-1}$

$p(x_t | z_{1:t-1}, u_{1:t}, m) = \sum_i {p(x_t | x^i_{t-1}, u_t, m) bel(x^i_{t-1})}$

Convolution of prior with transition model. Looking at the prior, this convolution yields the total probability that the vehicle started at the prior and wound up at $x_t$

#### Implementation Assumption
* Transition model is a 1D normal distribution.
* Transition model is independent of map $p(x_t | x_{t-1}, u_t, m) = p(x_t | x_{t-1}, u_t)$
* Transition model is defined by a 1D normal distribution with $\mu=u_t$, $\sigma = \sigma_{u_t}$ -> $N(x_t - x^i_{t-1}; u_t; \sigma_{u_t})$

In [8]:
// calculate probability of being at an estimated position at time t
float motion_model(float pseudo_position, float movement, vector<float> priors,
                  int map_size, int control_stdev) {
    float position_prob = 0.f;
    
    for(int i=0; i<map_size; i++) {
        float distance_ij = pseudo_position - i;
        float transition_prob = normpdf(distance_ij, movement, control_stdev);
        position_prob += transition_prob*priors[i];
    }
    
    return position_prob;
}

In [9]:
vector<float> motion_probs;
// calculate prob for each possible position
for(int i=0; i<map_size; i++) {
    motion_probs.push_back(motion_model(i, movement_per_timestep, 
                                       priors, map_size, control_stdev));
}
auto fig1 = plot(motion_probs);
fig1

A Jupyter widget

### Observation Model
$p(z_t | x_t, z_{1:t-1}, u_{1:t}, m) = p(z_t | x_t, m)$ (Markov Assumption)

Multiple observation model for each time step
$p(z_t | x_t, m) = p(z^1_t, ..., z^K_t | x_t, m)$

Since all observations are independent
$p(z^1_t, ..., z^K_t | x_t, m) = \prod^K_{k=1}{p(z^k_t | x_t, m)}$

Therefore
$p(z_t | x_t, z_{1:t-1}, u_{1:t}, m) = \prod^K_{k=1}{p(z^k_t | x_t, m)}$

#### Implementation Assumptions
* All observations are independent
* Range noise is Gaussian
* Only forward measurements are included
* Calculate psuedo ranges $z^*_t$ is estimated by $x_t$ and $m$. The psuedo range represent the range value if we are at $x_t$ in the map.
* $p(z^k_t | x_t, m) = N(z^k_t, z^{*k}_t, \sigma_{z_t})$

In [10]:
// pseudo range estimator function
auto pseudo_range_estimator(vector<float> landmark_positions, 
                            float pseudo_position) {
  // define pseudo observation vector
  vector<float> pseudo_ranges;
  
  // loop over number of landmarks and estimate pseudo ranges
  for(auto const& landmark : landmark_positions) {
    float dist = landmark - pseudo_position;
    if(dist > 0) {
      pseudo_ranges.push_back(dist);
    }
  }
  
  // sort pseudo range vector
  std::sort(pseudo_ranges.begin(), pseudo_ranges.end());
    
  return pseudo_ranges;
}

In [11]:
// step through each pseudo position x (i)
for (int i = 0; i < map_size; ++i) {
    float pseudo_position = float(i);
    // get pseudo ranges
    vector<float> pseudo_ranges = pseudo_range_estimator(landmark_positions, 
                                                         pseudo_position);
    // print to stdout
    if (pseudo_ranges.size() > 0) {
        for (int s = 0; s < pseudo_ranges.size(); ++s) {
            std::cout << "x: " << i << "\t" << pseudo_ranges[s] << std::endl;
        }
        std::cout << "-----------------------" << std::endl;
    }   
} 

x: 0	3
x: 0	9
x: 0	14
x: 0	23
-----------------------
x: 1	2
x: 1	8
x: 1	13
x: 1	22
-----------------------
x: 2	1
x: 2	7
x: 2	12
x: 2	21
-----------------------
x: 3	6
x: 3	11
x: 3	20
-----------------------
x: 4	5
x: 4	10
x: 4	19
-----------------------
x: 5	4
x: 5	9
x: 5	18
-----------------------
x: 6	3
x: 6	8
x: 6	17
-----------------------
x: 7	2
x: 7	7
x: 7	16
-----------------------
x: 8	1
x: 8	6
x: 8	15
-----------------------
x: 9	5
x: 9	14
-----------------------
x: 10	4
x: 10	13
-----------------------
x: 11	3
x: 11	12
-----------------------
x: 12	2
x: 12	11
-----------------------
x: 13	1
x: 13	10
-----------------------
x: 14	9
-----------------------
x: 15	8
-----------------------
x: 16	7
-----------------------
x: 17	6
-----------------------
x: 18	5
-----------------------
x: 19	4
-----------------------
x: 20	3
-----------------------
x: 21	2
-----------------------
x: 22	1
-----------------------


In [12]:
// calculates likelihood prob term based on landmark proximity for each pseudo position
float observation_model(vector<float> landmark_positions, 
                       vector<float> observations, float pseudo_position,
                       float observation_stdev) {
    float distance_prob = 1.0f;
    
    vector<float> pseudo_ranges = pseudo_range_estimator(landmark_positions, 
                                                         pseudo_position);
    // run over current observations
    for(auto const& observation:observations) {
        // define min distance
        float pseudo_range_min;
        
        if(pseudo_ranges.size() > 0) {
            // set min distance and remove from list
            pseudo_range_min = pseudo_ranges[0];
            pseudo_ranges.erase(pseudo_ranges.begin());
        } else { // no landmark so set a large distance
            pseudo_position = numeric_limits<const float>::infinity();
        }
        
        // estimate prob for observation model
        distance_prob *= normpdf(observation, pseudo_range_min, observation_stdev);
    }
    return distance_prob;
}

In [13]:
// define observations
vector<float> observation_probs;
// step through each pseudo position x (i)
for (int i = 0; i < map_size; ++i) {
    // get observation probability
    observation_probs.push_back(observation_model(landmark_positions, sensor_obs[5], 
                                                  i, observation_stdev));
}
for(auto const& ob:observation_probs) {
    std::cout << ob << std::endl;
}

0.000705351
0.000427817
1.29189e-05
2.39697e-12
8.70472e-08
0.000157385
0.0141673
0.0634936
0.0141673
2.18575e-15
1.45384e-12
4.81444e-11
7.93768e-11
6.51564e-12
4.50517e-24
7.42778e-24
6.09709e-25
2.49174e-27
5.06991e-31
5.13586e-36
2.5896e-42
0
0
0
0


In [14]:
auto fig2 = plot(normalize_vector(observation_probs));
fig2

A Jupyter widget

### Complete localization filter

In [15]:
// initial conditions are already set
// priors are already initialized in the notebook

vector<vector<float>> posterior_history;
posterior_history.push_back(priors);
vector<float> posterior(map_size, 0.0);
int time_steps = sensor_obs.size();

// cycle through each step
for(int t = 0; t < time_steps; ++t) {
    vector<float> observations;
    
    if(sensor_obs[t].empty()) {
        observations = {float(map_size)};
    } else {
        observations = sensor_obs[t];
    }
    
    // step through each psuedo position x(i)
    for(int i=0; i < map_size; ++i) {
        // motion prob
        float motion_prob = motion_model(i, movement_per_timestep, priors, map_size, 
                                         control_stdev);
        // observation prob
        float obs_prob = observation_model(landmark_positions, observations, i, 
                                           observation_stdev);
        // calculate ith posterior 
        posterior[i] = motion_prob * obs_prob;
    }
    posterior = normalize_vector(posterior);
    priors = posterior;
    posterior_history.push_back(posterior);
}

In [16]:
std::cout << posterior_history.size();

26

In [17]:
vector<xpl::figure> figures;
for(auto const& p :posterior_history) {
    figures.push_back(plot(p));
}

In [18]:
figures[0]

A Jupyter widget

In [19]:
figures[1]

A Jupyter widget

In [20]:
figures[2]

A Jupyter widget

In [21]:
figures[3]

A Jupyter widget

In [22]:
figures[4]

A Jupyter widget

In [23]:
figures[9]

A Jupyter widget

In [24]:
figures[20]

A Jupyter widget

In [28]:
figures[22]

A Jupyter widget